In [49]:
import pandas as pd
df=pd.read_csv('/content/diabetes.csv')

In [50]:
df.sample()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
542,10,90,85,32,0,34.9,0.825,56,1


In [51]:
x=df.iloc[:,:-1].values
y=df['Outcome'].values

In [52]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [53]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [54]:
import tensorflow as tf
from tensorflow.keras import layers

In [55]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [56]:
model=Sequential([
    Dense(28,activation='relu',input_dim=8),
    Dense(1, activation='sigmoid')
])

In [57]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [58]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
20/20 [==============================] - 2s 8ms/step - loss: 0.7149 - accuracy: 0.5049
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6444 - accuracy: 0.6336
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5958 - accuracy: 0.7036
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5614 - accuracy: 0.7362
Epoch 5/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5383 - accuracy: 0.7443
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5212 - accuracy: 0.7459
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5075 - accuracy: 0.7524
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4971 - accuracy: 0.7590
Epoch 9/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4891 - accuracy: 0.7606
Epoch 10/10
20/20 [==============================] - 0s 17ms/step - loss: 0.4826 - accuracy: 0.7638


##Tuning optimizer

In [59]:
def build_model(hp):
  model=Sequential([Dense(28,activation='relu',input_dim=8),Dense(1, activation='sigmoid')])
  optimizer=hp.Choice('optimizer',values=['adam','rmsprop','adadelta','sgd'])                #hp.choice for categorical, hp.int for numerical
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [65]:
!pip install keras_tuner
import keras_tuner as kt
tuner=kt.RandomSearch(build_model,objective='accuracy',max_trials=30,directory='hp_tuning',project_name='optimizer_tuning')

Reloading Tuner from hp_tuning/optimizer_tuning/tuner0.json


1.kt.RandomSearch:
RandomSearch is a method provided by Keras Tuner to perform random search for hyperparameter tuning. It randomly selects a combination of hyperparameters and evaluates the model's performance with those hyperparameters.

2.build_model:
This is a function you define to create and compile your Keras model. The RandomSearch tuner will call this function to build the model with different hyperparameter combinations.

3.objective='accuracy':
This specifies that the tuner should optimize for the 'accuracy' metric. You can choose other metrics depending on your specific problem, such as 'val_loss', 'val_accuracy', etc.

4.max_trials=6:
This sets the maximum number of different hyperparameter combinations that the tuner will try. In this case, it will test 6 different combinations.

5.directory='hp_tuning':
This specifies the directory where the tuner will save the results of the hyperparameter tuning process. It includes logs, checkpoints, and other relevant information.

6.project_name='optimizer_tuning':
This sets the name of the project, which is used to organize results within the specified directory. This can be useful if you are running multiple hyperparameter tuning projects and want to keep the results separate.

In [66]:
tuner.search(x_train, y_train, epochs=5)

In [67]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [63]:
model= tuner.get_best_models(num_models=1)[0] #we will selct best model and will save it in object of 'model'

In [68]:
model.summary()  # summary of the best model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 28)                252       
                                                                 
 dense_1 (Dense)             (None, 1)                 29        
                                                                 
Total params: 281 (1.10 KB)
Trainable params: 281 (1.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Tuning number of neurons in 1st hidden layer

In [71]:
def build_model(hp):
  neuron=hp.Int('no_neuron',min_value=4,max_value=128,step=4)
  model=Sequential([Dense(neuron,activation='relu',input_dim=8),Dense(1,activation='sigmoid')])
  model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')
  return model

In [72]:
tuner=kt.RandomSearch(build_model,objective='accuracy',max_trials=30,directory='hp_tuning',project_name='tuning_neurons')

In [73]:
tuner.search(x_train, y_train, epochs=5)

Trial 30 Complete [00h 00m 02s]
accuracy: 0.7622149586677551

Best accuracy So Far: 0.7850162982940674
Total elapsed time: 00h 00m 58s


In [74]:
tuner.get_best_hyperparameters()[0].values

{'no_neuron': 76}

In [75]:
model = tuner.get_best_models(num_models=1)[0]

In [76]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 76)                684       
                                                                 
 dense_1 (Dense)             (None, 1)                 77        
                                                                 
Total params: 761 (2.97 KB)
Trainable params: 761 (2.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Tuning number of layers & optimizer

In [77]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(112, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'adadelta'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [79]:
tuner = kt.RandomSearch(build_model,
                        objective='accuracy',
                        max_trials=30,
                        directory='hp_tuning',
                        project_name='No_layers')

In [80]:
tuner.search(x_train, y_train, epochs=5)

Trial 30 Complete [00h 00m 03s]
accuracy: 0.30293160676956177

Best accuracy So Far: 0.8029316067695618
Total elapsed time: 00h 01m 28s


In [81]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7, 'optimizer': 'adam'}

In [82]:
model = tuner.get_best_models(num_models=1)[0]

In [83]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               1008      
                                                                 
 dense_1 (Dense)             (None, 32)                3616      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 32)                1

### Tuning Activation and number of neurons

In [84]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  hp_activation = hp.Choice('activation', values=['relu', 'tanh'])                        #activation only once
  layer1_unit = hp.Int('neurons_layer1', min_value=1, max_value=1000, step=100)           # for layer 1
  layer2_unit = hp.Int('neurons_layer2', min_value=1, max_value=1000, step=100)           # for layer 2
  model.add(Dense(layer1_unit, activation=hp_activation))
  model.add(Dense(layer2_unit, activation=hp_activation))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [86]:
tuner = kt.RandomSearch(build_model,
                        objective='accuracy',
                        max_trials=30,
                        directory='Dir',
                        project_name='second_exp')

In [87]:
tuner.search(x_train, y_train, epochs=5)

Trial 30 Complete [00h 00m 02s]
accuracy: 0.653094470500946

Best accuracy So Far: 0.8110749125480652
Total elapsed time: 00h 01m 35s


In [88]:
tuner.get_best_hyperparameters()[0].values

{'activation': 'relu', 'neurons_layer1': 401, 'neurons_layer2': 801}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

In [90]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 401)               13233     
                                                                 
 dense_2 (Dense)             (None, 801)               322002    
                                                                 
 dense_3 (Dense)             (None, 1)                 802       
                                                                 
Total params: 336325 (1.28 MB)
Trainable params: 336325 (1.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Tuning Number of layers

In [91]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(8, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [92]:
tuner = kt.RandomSearch(build_model,
                        objective='accuracy',
                        max_trials=30,
                        directory='Dir',
                        project_name='third_exp')

In [93]:
tuner.search(x_train, y_train, epochs=5)

Trial 9 Complete [00h 00m 03s]
accuracy: 0.7214983701705933

Best accuracy So Far: 0.7687296271324158
Total elapsed time: 00h 00m 35s


In [94]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [95]:
model = tuner.get_best_models(num_models=1)[0]

In [97]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 32)                288       
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 32)                1

#Tuning Number of layers and number of neurons

In [98]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(8, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(hp.Int(f'neurons{i}', min_value=4, max_value=64, step=4),
                    activation=hp.Choice(f'activation{i}', values=['tanh', 'relu'])))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [99]:
tuner = kt.RandomSearch(build_model,
                        objective='accuracy',
                        max_trials=30,
                        directory='Dir',
                        project_name='fifth_exp')

In [100]:
tuner.search(x_train, y_train, epochs=5)

Trial 30 Complete [00h 00m 05s]
accuracy: 0.7654722929000854

Best accuracy So Far: 0.7833876013755798
Total elapsed time: 00h 01m 55s


In [101]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'neurons0': 20,
 'activation0': 'relu',
 'neurons1': 40,
 'activation1': 'relu',
 'neurons2': 64,
 'activation2': 'relu',
 'neurons3': 52,
 'activation3': 'tanh',
 'neurons4': 16,
 'activation4': 'tanh',
 'neurons5': 64,
 'activation5': 'tanh',
 'neurons6': 36,
 'activation6': 'tanh',
 'neurons7': 44,
 'activation7': 'relu',
 'neurons8': 16,
 'activation8': 'tanh',
 'neurons9': 4,
 'activation9': 'tanh'}

In [102]:
model = tuner.get_best_models(num_models=1)[0]

In [103]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 20)                180       
                                                                 
 dense_2 (Dense)             (None, 40)                840       
                                                                 
 dense_3 (Dense)             (None, 64)                2624      
                                                                 
 dense_4 (Dense)             (None, 52)                3380      
                                                                 
 dense_5 (Dense)             (None, 16)                848       
                                                                 
 dense_6 (Dense)             (None, 64)                1

##Putting it all togather

In [104]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(8, activation='relu', input_dim=8))

  # unit = hp.Int('neurons', min_value=4, max_value=64, step=4)
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(hp.Int(f'neurons{i}', min_value=4, max_value=64, step=4),
                    activation=hp.Choice(f'activation{i}', values=['tanh', 'relu'])))

  model.add(Dense(1, activation='sigmoid'))


  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [105]:
tuner = kt.RandomSearch(build_model,
                        objective='accuracy',
                        max_trials=30,
                        directory='Dir',
                        project_name='sixth_exp')

In [106]:
tuner.search(x_train, y_train, epochs=5)

Trial 30 Complete [00h 00m 03s]
accuracy: 0.732899010181427

Best accuracy So Far: 0.7850162982940674
Total elapsed time: 00h 01m 53s


In [108]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'neurons0': 40,
 'activation0': 'relu',
 'optimizer': 'adam',
 'neurons1': 44,
 'activation1': 'relu',
 'neurons2': 52,
 'activation2': 'tanh',
 'neurons3': 44,
 'activation3': 'tanh',
 'neurons4': 12,
 'activation4': 'tanh',
 'neurons5': 4,
 'activation5': 'tanh',
 'neurons6': 32,
 'activation6': 'relu',
 'neurons7': 60,
 'activation7': 'tanh',
 'neurons8': 44,
 'activation8': 'relu',
 'neurons9': 40,
 'activation9': 'relu'}

In [107]:
model = tuner.get_best_models(num_models=1)[0]

In [109]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 40)                360       
                                                                 
 dense_2 (Dense)             (None, 44)                1804      
                                                                 
 dense_3 (Dense)             (None, 52)                2340      
                                                                 
 dense_4 (Dense)             (None, 44)                2332      
                                                                 
 dense_5 (Dense)             (None, 12)                540       
                                                                 
 dense_6 (Dense)             (None, 4)                 5

In [110]:
#training after 5 epochs so that it will retain the performance which it did before, if we start from beginning, it may override and overfit
model.fit(x_train, y_train, epochs=100, initial_epoch=5)

Epoch 6/100
20/20 [==============================] - 3s 4ms/step - loss: 0.4741 - accuracy: 0.7736
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4653 - accuracy: 0.7850
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4582 - accuracy: 0.7932
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4561 - accuracy: 0.7915
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4502 - accuracy: 0.7964
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4445 - accuracy: 0.7980
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4400 - accuracy: 0.7932
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4307 - accuracy: 0.8094
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4261 - accuracy: 0.8013
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4158 - accuracy: 0.8062
Epoc